In [ ]:
python src/tools/image_slicer.py --image_path 2025_08_05_PhotoRieboR4_g201b201078_f003_259.JPG --overlap_percentage 15 --destination_folder ./tiles/ --tile-size 1024

In [40]:
tile_path = "tiles/2025_08_05_PhotoRieboR4_g201b201078_f003_259_tile_007_008.png"

In [43]:
from ultralytics import YOLO

model = YOLO("yolov8x.pt")

In [41]:
# How to specify the threshold for the predictions?
predictions = model.predict(tile_path, conf=0.1)
print(len(predictions[0].boxes.conf))


image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\tiles\2025_08_05_PhotoRieboR4_g201b201078_f003_259_tile_007_008.png: 640x640 7 birds, 632.7ms
Speed: 3.9ms preprocess, 632.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
7


In [ ]:
predictions[0].save("predicted_image.png")

'predicted_image.png'

In [17]:
images_folder = "tiles/"

In [45]:
import os
output_folder = "predicted_images-yolov8x"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for image_path in os.listdir(images_folder):
    predictions = model.predict(f"{images_folder}/{image_path}", conf=0.1)
    if len(predictions[0].boxes.conf) > 0:
        predictions[0].save(f"{output_folder}/{image_path}")



image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\tiles\2025_08_05_PhotoRieboR4_g201b201078_f003_259_tile_000_000.png: 640x640 (no detections), 656.1ms
Speed: 3.3ms preprocess, 656.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\tiles\2025_08_05_PhotoRieboR4_g201b201078_f003_259_tile_000_001.png: 640x640 (no detections), 753.1ms
Speed: 4.5ms preprocess, 753.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\tiles\2025_08_05_PhotoRieboR4_g201b201078_f003_259_tile_000_002.png: 640x640 (no detections), 744.6ms
Speed: 3.1ms preprocess, 744.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\tiles\2025_08_05_PhotoRieboR4_g201b201078_f003_259_tile_000_003.png: 640x640 (no det

In [ ]:
import multiprocessing as mp
from functools import partial
from tqdm import tqdm
import time

def process_single_image(image_path, images_folder, model_path="yolo11x.pt", conf=0.1):
    """
    Обрабатывает одно изображение с помощью YOLO модели
    """
    try:
        from ultralytics import YOLO
        model = YOLO(model_path)
        
        full_path = f"{images_folder}/{image_path}"
        predictions = model.predict(full_path, conf=conf, verbose=False)
        
        if len(predictions[0].boxes.conf) > 0:
            output_path = f"predicted_images/{image_path}"
            predictions[0].save(output_path)
            return image_path, len(predictions[0].boxes.conf), True
        else:
            return image_path, 0, False
    except Exception as e:
        print(f"Ошибка при обработке {image_path}: {e}")
        return image_path, 0, False


In [ ]:
def process_images_multiprocessing(images_folder="tiles/", num_processes=None, conf=0.1):
    """
    Обрабатывает изображения с использованием многопроцессорности
    """
    # Создаём папку для результатов если её нет
    if not os.path.exists("predicted_images"):
        os.makedirs("predicted_images")
    
    # Получаем список всех изображений
    image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    
    print(f"Найдено {total_images} изображений для обработки")
    
    if num_processes is None:
        num_processes = min(mp.cpu_count(), 8)  # Ограничиваем максимум 8 процессами
    
    print(f"Используется {num_processes} процессов")
    
    # Создаём частичную функцию с зафиксированными параметрами
    process_func = partial(process_single_image, 
                          images_folder=images_folder, 
                          conf=conf)
    
    start_time = time.time()
    
    # Многопроцессорная обработка
    with mp.Pool(processes=num_processes) as pool:
        results = list(tqdm(pool.imap(process_func, image_files), 
                           total=total_images, 
                           desc="Обработка изображений"))
    
    end_time = time.time()
    
    # Анализ результатов
    processed_count = sum(1 for _, _, saved in results if saved)
    total_detections = sum(count for _, count, _ in results)
    
    print(f"\nРезультаты обработки:")
    print(f"Всего изображений: {total_images}")
    print(f"Изображений с детекциями: {processed_count}")
    print(f"Общее количество детекций: {total_detections}")
    print(f"Время обработки: {end_time - start_time:.2f} секунд")
    print(f"Среднее время на изображение: {(end_time - start_time) / total_images:.2f} секунд")
    
    return results


In [ ]:
# Запускаем многопроцессорную обработку
if __name__ == "__main__":  # Важно для Windows
    results = process_images_multiprocessing(
        images_folder="tiles/",
        num_processes=4,  # Можете изменить количество процессов
        conf=0.1
    )
